# Langages de script – Python

## Modèle vectoriel

### M2 Ingénierie Multilingue – INaLCO

clement.plancq@ens.fr

## Modélisation vectorielle des documents
Modèle symbolique : un document est une suite de symboles (mots)

In [ ]:
with open("zola_ventre-de-paris.txt") as in_stream:
    doc = [word for line in in_stream for word in line.strip().split()]
doc[5:10] 

  - Quand on peut/veut écrire des règles de traitement à la main,  c'est pratique et Unitex est votre ami
  - Pour faire de l'apprentissage/non-supervisé, on ne sait pas bien le gérer
    - Parce que l'apprentissage artificiel c'est des maths, et que les maths discrètes c'est dur™

Ce qu'on sait bien gérer : les nombres !

  - On va donc plutôt représenter nos documents par des nombres
  - Ou plutôt des suites de nombres
  
Le problème est donc : comment représenter des documents par une suite de nombres en conservant les informations qu'on veut exploiter ?

## Les sacs de mots

Idée : représenter un document par la distribution brute de son lexique (le nombre d'occurences de chaque mot)

  - On l'a déjà fait dans [le TP vocabulaire commun](voc-commun.ipynb)

In [ ]:
from collections import Counter
with open("zola_ventre-de-paris.txt") as in_stream:
    doc1 = Counter(word for line in in_stream for word in line.strip().split())
doc1

  - C'est assez primitif, mais ça marche étonnamment bien
  - Pour pouvoir faire des comparaisons entre documents il faut utiliser le même lexique pour tous

In [ ]:
with open("balzac_petits-bourgeois.txt") as in_stream:
    doc2 = Counter(word for line in in_stream for word in line.strip().split())

voc = sorted(set(doc1.keys()).union(doc2.keys()))
vec_1 = [doc1[word] for word in voc]
vec_2 = [doc2[word] for word in voc]

vec_1

Ici `vec_1` et `vec_2` sont des listes des nombres (des *vecteurs*) de même longueur auxquelles on va pouvoir appliquer les merveilleux outils que nous offre la statistique.

### Exercice
  - Écrire un script qui prend en entrée un dossier contenant des documents (sous forme de fichier textes) et sort un fichier TSV donnant pour chaque document sa représentation en sac de mots (en nombre d'occurences des mots du vocabulaire commun)
    - Dans le sens habituel : un fichier par ligne, un mot par colonne
    - Pour itérer sur les fichiers dans un dossier on peut utiliser `for f in pathlib.Path(chemin_du_dossier).glob('*')`
    - Pour récupérer des arguments en ligne de commande : [`argparse`](https://docs.python.org/3/library/argparse.html) ou [`sys.argv`](https://docs.python.org/3/library/argparse.html)
  - Tester sur la partie positive du [mini-corpus imdb](imdb_smol.tar.gz)

On pourrait déjà se servir de ce genre de représentation pour faire des travaux intéressants, au hasard de la classification de documents, pourquoi pas avec des SVMs…

## Listes de nombres
### Opérations sur les lignes
Les sacs de mots en nombre d'occurences c'est bien gentil, mais si on a des documents de tailles homogènes on a vite des problèmes

In [ ]:
# Quick 'n dirty BOWization
import re
import math

def to_bow(texts):
    freqs = [Counter(re.split(r'\W+', t.lower())) for t in texts]
    voc = sorted(set().union(*[c.keys() for c in freqs]))
    return [[c[word] for word in voc] for c in freqs]

def l2(b1, b2):
    return math.sqrt(sum((x-y)**2 for x, y in zip(b1, b2)))


In [ ]:
d1 = "Un crocodile"
d2 = "Un crocodile, un crocodile, un crocodile"
d3 = "Un éléphant"

b1, b2, b3 = to_bow([d1, d2, d3])

print(
    '\n'.join('\t'.join([str(x) for x in l]) for l in [b1, b2, b3])
)

print(l2(b1, b2))
print(l2(b1, b3))


Heureusement ce n'est pas très compliqué de normaliser pour avoir des fréquences relatives

In [ ]:
n1 = [x/sum(b1) for x in b1]
n2 = [x/sum(b2) for x in b2]
n3 = [x/sum(b3) for x in b3]

print(l2(n1, n2))
print(l2(n1, n3))
print(n1, n2, n3)

#### Exercice
Modifier le script précédent pour qu'il génère des sacs de mots utilisant les fréquences relatives plutot que les nombres d'occurences

## Opérations sur les colonnes
  - Fréquences relatives : sont des bons indicateurs du contenu lexical d'**un** texte
  - Mais pas forcément suffisant pour de la fouille de corpus
  - On a besoin d'indicateurs relatifs à l'ensemble des textes

Par exemple : un indicateur de spécificité assez grossier d'un mot $W$ dans un texte $T$.
$$
S(W, T) = \frac{\text{nombre d'occurrences de W dans le texte T}}{\text{nombre d'occurences de W dans l'ensemble du corpus}}
$$

Le calcul pour un seul mot n'est pas beaucoup plus compliqué que le calcul des fréquences relatives

In [ ]:
d1 = "Un crocodile"
d2 = "Un crocodile, un crocodile, un crocodile"
d3 = "Un éléphant"

bows = to_bow([d1, d2, d3])
total_crocodile = sum(l[0] for l in bows)
[l[0]/total_crocodile for l in bows]

  - Si on veut le faire pour tous les mots c'est un peu plus pénible à écrire
  - Et encore : c'est une des opérations les plus simples !

Une solution c'est de voir l'ensemble des sacs-de-mots non plus comme une liste de listes, mais comme un tableau à deux dimensions

In [ ]:
print('\n'.join('  '.join([str(x) for x in l]) for l in bows))

Avec cette vision, les deux opérations sont en fait quasiment identiques

  - Pour calculer les fréquences relatives, on divise chaque ligne par sa somme
  - Pour calculer les spécificités, on divise chaque colonne par sa somme
  
Il ne nous manque plus qu'une interface agréable pour le faire

## Numpy à la rescousse


In [ ]:
import numpy as np

[Numpy](http://numpy.org) est une bibliothèque de calcul numérique pour Python. Elle est à la base de nombreux autres (dont [`scikit-learn`](https://scikit-learn.org) et [`gensim`](https://radimrehurek.com/gensim) dont on reparlera) et est assez incontournable si on veut faire du calcul en Python.

Ses possibilités sont très nombreuses, mais on se contentra ici de ses fonctions de base : la manipulation de tableaux de nombres.

### `numpy.array`

La classe de base de numpy est [`numpy.ndarray`](https://docs.scipy.org/doc/numpy/reference/arrays.ndarray.html), qui permet de représenter des tableaux de nombres de dimensions arbitraires (chez les informaticiens on appelle ça des **tenseurs**, ce qui fait ricaner les mathématiciens).

On les créé en général avec la fonction [`numpy.array`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.array.html)

In [ ]:
a = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
a

Il y a [beaucoup de façons de créer des `ndarray`s](https://docs.scipy.org/doc/numpy/reference/routines.array-creation.html#routines-array-creation). Comme toujours, il est recommandé de jeter un œil sur [la doc](https://docs.scipy.org/doc/numpy)

In [ ]:
a

Contrairement à nos listes de listes précédentes, où on n'a accès qu'à une dimension à la fois, ici on peut accéder librement aux deux dimensions

In [ ]:
print(a[0, 0])
print(a[2, 1])

Par convention, les coordonnées sont données dans l'ordre $(\text{ligne}, \text{colonne})$.

In [ ]:
a

On peut aussi — comme pour les listes habituelles de Python — adresser des tranches

In [ ]:
a[0, 1:3]

In [ ]:
a[:2, 1]

In [ ]:
a[1:3, :2]

In [ ]:
a[0, :]

In [ ]:
a

Le tableau reste quand même une séquence indexable, sur laquelle on peut itérer, dans l'ordre de ses dimensions

In [ ]:
a[0][1]  # Strictement équivalent à `a[0, 1]

In [ ]:
for l in a:
    print(l)
    for x in l:
        print(x)

### Manipulations de tableaux
Quelques fonctions bien pratiques

In [ ]:
a

In [ ]:
a.max()

In [ ]:
a.min()

In [ ]:
a.mean()

In [ ]:
a.sum()

In [ ]:
a.transpose()

In [ ]:
a.sum(axis=0)

### Opérations
Les opérations habituelles sont définies et ont le sens qu'on a envie qu'elles aient

In [ ]:
a

In [ ]:
a*2

In [ ]:
a/3

In [ ]:
a + 4

In [ ]:
a - 5

In [ ]:
a

Les opérations sont aussi définies entre deux tableaux

In [ ]:
b = np.array([[1, 2, 3], [4, 5, 6], [0, 0, 0]])
b

In [ ]:
a+b

In [ ]:
a*b

In [ ]:
a/b

### Broadcasting
Une notion un peu plus compliquée mais qui va nous servir tout de suite

In [ ]:
a

In [ ]:
c = np.array([2, 4, 8])
c

In [ ]:
a*c

Explication : si un des tableaux a moins de dimensions que l'autre, numpy fait automatiquement la conversion pour que tout se passe comme si on avait multiplié par

In [ ]:
np.broadcast_to(c, [3,3])

Multiplier par un tableau à une dimension revient donc à multiplier colonne par colonne

## Revenons à nos moutons
Rappel : on avait une liste de sacs de mots et on voulait obtenir un tableau de spécificités

In [ ]:
bows

Il suffit de sommer chaque colonne

In [ ]:
bows_array = np.array(bows)
cols_total = bows_array.sum(axis=0)
print(bows_array)
print(cols_total)

Puis de diviser

In [ ]:
bows_array/cols_total

## Exercice
Modifier le script de BoWization précédent pour qu'il renvoie non plus les fréquences relatives de chaque mot mais leur tf⋅idf avec la définition suivante pour un mot $w$, un document $D$ et un corpus $C$

  - $\mathrm{tf}(w, D)$ est la fréquence relative de $w$ dans $D$
  - $$\mathrm{idf}(w, C) = \log\!\left(\frac{\text{nombre de documents dans $C$}}{\text{nombre de documents de $C$ qui contiennent $w$}}\right)$$
  - $\log$ est le logarithme naturel [`np.log`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.log.html)
  - $\mathrm{tfidf}(w, D, C) = \mathrm{tf}(w, D)×\mathrm{idf}(w, C)$
  
Pistes de recherche:
  - L'option `keepdims` de `np.sum`
  - `np.transpose`
  - `np.count_nonzero`
  - Regarder ce que donne `np.array([[1, 0], [2, 0]]) > 0`

## Représentations denses de documents
Dans les représentations qu'on a vu, les vecteurs qui composent chaque texte sont essentiellement composés de zéros
  
  - On dit qu'il s'agit de vecteurs *creux*
  - Ils sont aussi de très haute dimensions : il y a en général plus de mots que de documents
  - Merci la loi de Zipf
  
Pour certaines applications ce n'est pas un problème
  - Les classifieurs SVM et *Random Forest* fonctionnent bien avec ce genre de données

Mais pour les réseaux de neurones, par exemple, c'est très peu efficace.

Il y a plusieurs autres solutions:

  - Doc2vec et al. → voir [gensim](https://radimrehurek.com/gensim/models/doc2vec.html)
  - Utiliser des techniques de réduction de dimension → voir [le notebook visualisation](visualisation.ipynb)